In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [3]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [4]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [5]:
data = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_concat_countries_and_wb_groups_income.pkl')#.drop(['Addresses'], axis=1)

data.head(2)

,wos_ID,Addresses,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,...,wb_lower_middle_income_economies,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others,funding_yes,open_access_yes
0,wos:000904661800001,"[Bi, Jinhao; Xia, Xianzhu] Jilin Agr Univ, Col...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0,1
1,wos:000921279800001,"[Ye, Xin; Holland, Richard; Wood, Mark; Pasetk...",NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1,1


In [6]:
data_topics = pd.read_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_prep_lemmatized_topics.pkl')[['Topic N°']].reset_index()
data_topics.head(2)

,wos_ID,Topic N°
0,wos:000904661800001,3
1,wos:000921279800001,5


In [7]:
data_and_topics_number = pd.concat([data.set_index('wos_ID'), 
                                   data_topics.set_index('wos_ID')], 
                                  axis=1)
data_and_topics_number.head(2)

,Addresses,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,...,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others,funding_yes,open_access_yes,Topic N°
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000904661800001,"[Bi, Jinhao; Xia, Xianzhu] Jilin Agr Univ, Col...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0,1,3
wos:000921279800001,"[Ye, Xin; Holland, Richard; Wood, Mark; Pasetk...",NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1,1,5


# The topics of Marburg Virus Diseases publications 

## proportion all

In [8]:
countries_topics_list_all = data_and_topics_number.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_all.rename(columns = {0:'all_is_no_oa', 
#                               1:'all_is_oa'}, inplace = True)

countries_topics_list_all['all_total'] = countries_topics_list_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_all.columns = countries_topics_list_all.columns.astype(str)

for col in countries_topics_list_all.columns:
    countries_topics_list_all[col + '_%all'] = countries_topics_list_all[col]/countries_topics_list_all['all_total']*100
    
# save data
# countries_topics_list_all.to_csv(f'../TABLES/countries_all_oa_list.csv')
countries_topics_list_all

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%all,2_%all,3_%all,4_%all,5_%all,6_%all,7_%all,8_%all,9_%all,all_total_%all
wos_ID,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0
Addresses,50,78,58,78,51,127,90,161,95,88,...,8.904110,6.621005,8.904110,5.821918,14.497717,10.273973,18.378995,10.844749,10.045662,100.0
Afghanistan,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
United States,30,48,31,69,39,98,65,105,42,59,...,8.191126,5.290102,11.774744,6.655290,16.723549,11.092150,17.918089,7.167235,10.068259,100.0
Angola,0,0,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_others,3,7,9,4,5,11,15,21,6,6,...,8.045977,10.344828,4.597701,5.747126,12.643678,17.241379,24.137931,6.896552,6.896552,100.0
wb_high_income_economies,42,76,52,74,50,125,80,152,86,83,...,9.268293,6.341463,9.024390,6.097561,15.243902,9.756098,18.536585,10.487805,10.121951,100.0
wb_high_income_economies_others,42,76,52,74,50,125,80,152,86,83,...,9.268293,6.341463,9.024390,6.097561,15.243902,9.756098,18.536585,10.487805,10.121951,100.0
funding_yes,66,79,59,79,51,141,91,170,107,89,...,8.476395,6.330472,8.476395,5.472103,15.128755,9.763948,18.240343,11.480687,9.549356,100.0


In [9]:
# countries_topics_list_all.describe()

## proportion fund

In [10]:
data_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 1]
data_fund.shape

(577, 122)

In [11]:
countries_topics_list_fund = data_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_fund.rename(columns = {0:'fund_is_no_oa', 
#                               1:'fund_is_oa'}, inplace = True)

countries_topics_list_fund['fund_total'] = countries_topics_list_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_fund.columns = countries_topics_list_fund.columns.astype(str)

for col in countries_topics_list_fund.columns:
    countries_topics_list_fund[col+'_%fund'] = countries_topics_list_fund[col]/countries_topics_list_fund['fund_total']*100

# save data
# countries_topics_list_fund.to_csv(f'../TABLES/countries_fund_oa_list.csv')

countries_topics_list_fund

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%fund,2_%fund,3_%fund,4_%fund,5_%fund,6_%fund,7_%fund,8_%fund,9_%fund,fund_total_%fund
wos_ID,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
Addresses,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United States,10,38,27,49,34,81,57,62,25,51,...,8.755760,6.221198,11.290323,7.834101,18.663594,13.133641,14.285714,5.760369,11.751152,100.0
Angola,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_others,1,4,7,3,5,9,15,13,4,5,...,6.060606,10.606061,4.545455,7.575758,13.636364,22.727273,19.696970,6.060606,7.575758,100.0
wb_high_income_economies,11,53,35,54,43,94,68,82,42,64,...,9.706960,6.410256,9.890110,7.875458,17.216117,12.454212,15.018315,7.692308,11.721612,100.0
wb_high_income_economies_others,11,53,35,54,43,94,68,82,42,64,...,9.706960,6.410256,9.890110,7.875458,17.216117,12.454212,15.018315,7.692308,11.721612,100.0
funding_yes,12,55,40,56,44,95,77,87,43,68,...,9.532062,6.932409,9.705373,7.625650,16.464471,13.344887,15.077990,7.452340,11.785095,100.0


## proportion no fund

In [12]:
data_no_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0]
data_no_fund.head(2)

,Addresses,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,Belize,...,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others,funding_yes,open_access_yes,Topic N°
wos_ID,,,,,,,,,,,,,,,,,,,,,
wos:000904661800001,"[Bi, Jinhao; Xia, Xianzhu] Jilin Agr Univ, Col...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0,1,3
wos:000849602200001,"[Sah, Ranjit] Tribhuvan Univ Teaching Hosp, In...",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,NaN,NaN,NaN,1.0,1.0,0,1,8


In [13]:
countries_topics_list_no_fund = data_no_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_no_fund.rename(columns = {0:'no_fund_is_no_oa', 
#                               1:'no_fund_is_oa'}, inplace = True)

countries_topics_list_no_fund['no_fund_total'] = countries_topics_list_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_no_fund.columns = countries_topics_list_no_fund.columns.astype(str)

for col in countries_topics_list_no_fund.columns:
    countries_topics_list_no_fund[col + '_%no_fund'] = countries_topics_list_no_fund[col]/countries_topics_list_no_fund['no_fund_total']*100
    
# countries_topics_list_no_fund['%_diff(%no_fund_is_oa-%no_fund_is_no_oa)'] = countries_topics_list_no_fund['%no_fund_is_oa'] - countries_topics_list_no_fund['%no_fund_is_no_oa']

# save data
# countries_topics_list_no_fund.to_csv(f'../TABLES/countries_fund_no_oa_list.csv')

countries_topics_list_no_fund

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
wos_ID,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Addresses,38,23,18,22,7,32,13,74,52,20,...,7.692308,6.020067,7.357860,2.341137,10.702341,4.347826,24.749164,17.391304,6.688963,100.0
Afghanistan,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
United States,20,10,4,20,5,17,8,43,17,8,...,6.578947,2.631579,13.157895,3.289474,11.184211,5.263158,28.289474,11.184211,5.263158,100.0
Angola,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_others,2,3,2,1,0,2,0,8,2,1,...,14.285714,9.523810,4.761905,0.000000,9.523810,0.000000,38.095238,9.523810,4.761905,100.0
wb_high_income_economies,31,23,17,20,7,31,12,70,44,19,...,8.394161,6.204380,7.299270,2.554745,11.313869,4.379562,25.547445,16.058394,6.934307,100.0
wb_high_income_economies_others,31,23,17,20,7,31,12,70,44,19,...,8.394161,6.204380,7.299270,2.554745,11.313869,4.379562,25.547445,16.058394,6.934307,100.0
funding_yes,54,24,19,23,7,46,14,83,64,21,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0


## concat all dataframe and save

In [14]:
# concatenate dataframes on index
concat_countries_oa_list = pd.concat([countries_topics_list_all, 
                                        countries_topics_list_fund, 
                                        countries_topics_list_no_fund], 
                                       axis=1)
# save data
concat_countries_oa_list.to_csv(f'../TABLES/concat_countries_topics_list.csv')
concat_countries_oa_list

Topic N°,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
wos_ID,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0
Addresses,50,78,58,78,51,127,90,161,95,88,...,7.692308,6.020067,7.357860,2.341137,10.702341,4.347826,24.749164,17.391304,6.688963,100.0
Afghanistan,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
United States,30,48,31,69,39,98,65,105,42,59,...,6.578947,2.631579,13.157895,3.289474,11.184211,5.263158,28.289474,11.184211,5.263158,100.0
Angola,0,0,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_others,3,7,9,4,5,11,15,21,6,6,...,14.285714,9.523810,4.761905,0.000000,9.523810,0.000000,38.095238,9.523810,4.761905,100.0
wb_high_income_economies,42,76,52,74,50,125,80,152,86,83,...,8.394161,6.204380,7.299270,2.554745,11.313869,4.379562,25.547445,16.058394,6.934307,100.0
wb_high_income_economies_others,42,76,52,74,50,125,80,152,86,83,...,8.394161,6.204380,7.299270,2.554745,11.313869,4.379562,25.547445,16.058394,6.934307,100.0
funding_yes,66,79,59,79,51,141,91,170,107,89,...,6.760563,5.352113,6.478873,1.971831,12.957746,3.943662,23.380282,18.028169,5.915493,100.0


## significance

In [15]:
for topics in range(10):
    fund_topics_percent = data_and_topics_number.loc[data_and_topics_number['Topic N°'] == topics]['funding_yes'].value_counts(normalize=True)[1]
    
    print(f'Funding percentage Topic {topics} : 'f'{fund_topics_percent}')

Funding percentage Topic 0 : 0.18181818181818182
Funding percentage Topic 1 : 0.6962025316455697
Funding percentage Topic 2 : 0.6779661016949152
Funding percentage Topic 3 : 0.7088607594936709
Funding percentage Topic 4 : 0.8627450980392157
Funding percentage Topic 5 : 0.6737588652482269
Funding percentage Topic 6 : 0.8461538461538461
Funding percentage Topic 7 : 0.5117647058823529
Funding percentage Topic 8 : 0.40186915887850466
Funding percentage Topic 9 : 0.7640449438202247


In [16]:
# fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 1) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']
# no_fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 0) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']

In [17]:
# test_sig(fund_topics, no_fund_topics)

# Topics Trends

In [18]:
data_and_topics_number2 = data_and_topics_number[['Publication Year','funding_yes','Topic N°']].reset_index().set_index(['wos_ID', 'Publication Year','funding_yes'])
data_and_topics_number2

,,,Topic N°
wos_ID,Publication Year,funding_yes,
wos:000904661800001,2023.0,0,3
wos:000921279800001,2023.0,1,5
wos:000909001200001,2023.0,1,7
wos:000777045900001,2022.0,1,8
wos:000849602200001,2022.0,0,8
...,...,...,...
wos:a1968b358900049,1968.0,0,5
wos:a1968a855300001,1968.0,0,8
wos:a1968a855300006,1968.0,0,8


In [19]:
data_topics_trend = pd.get_dummies(data_and_topics_number2['Topic N°'])
data_topics_trend

,,,0,1,2,3,4,5,6,7,8,9
wos_ID,Publication Year,funding_yes,,,,,,,,,,
wos:000904661800001,2023.0,0,0,0,0,1,0,0,0,0,0,0
wos:000921279800001,2023.0,1,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,2023.0,1,0,0,0,0,0,0,0,1,0,0
wos:000777045900001,2022.0,1,0,0,0,0,0,0,0,0,1,0
wos:000849602200001,2022.0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
wos:a1968b358900049,1968.0,0,0,0,0,0,0,1,0,0,0,0
wos:a1968a855300001,1968.0,0,0,0,0,0,0,0,0,0,1,0
wos:a1968a855300006,1968.0,0,0,0,0,0,0,0,0,0,1,0


## ALL

In [20]:
data_topics_trend_all = data_topics_trend.groupby('Publication Year').sum()

data_topics_trend_all['all_total'] = data_topics_trend_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_all.columns = data_topics_trend_all.columns.astype(str)

for col in data_topics_trend_all.columns:
    data_topics_trend_all[col + '_%all'] = data_topics_trend_all[col]/data_topics_trend_all['all_total']*100

data_topics_trend_all

,0,1,2,3,4,5,6,7,8,9,...,1_%all,2_%all,3_%all,4_%all,5_%all,6_%all,7_%all,8_%all,9_%all,all_total_%all
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0


## fund

In [21]:
data_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==1].set_index(['wos_ID','funding_yes'])
data_fund.head(2)

,,Publication Year,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,,,,,,,,,,,
wos:000921279800001,1,2023.0,0,0,0,0,0,1,0,0,0,0
wos:000909001200001,1,2023.0,0,0,0,0,0,0,0,1,0,0


In [22]:
data_topics_trend_fund = data_fund.groupby('Publication Year').sum()

data_topics_trend_fund['fund_total'] = data_topics_trend_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_fund.columns = data_topics_trend_fund.columns.astype(str)

for col in data_topics_trend_fund.columns:
    data_topics_trend_fund[col + '_%fund'] = data_topics_trend_fund[col]/data_topics_trend_fund['fund_total']*100
    

data_topics_trend_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%fund,2_%fund,3_%fund,4_%fund,5_%fund,6_%fund,7_%fund,8_%fund,9_%fund,fund_total_%fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
2002.0,0,0,0,0,0,0,1,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,100.0
2003.0,0,1,0,0,0,0,0,0,0,0,...,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
2004.0,0,0,0,0,0,0,0,1,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,100.0
2005.0,0,0,0,0,0,0,2,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,100.0
2006.0,0,0,0,0,2,3,0,0,0,0,...,0.000000,0.000000,0.000000,40.000000,60.000000,0.000000,0.000000,0.000000,0.000000,100.0
2007.0,0,2,2,0,0,2,1,0,0,3,...,20.000000,20.000000,0.000000,0.000000,20.000000,10.000000,0.000000,0.000000,30.000000,100.0
2008.0,0,0,1,3,0,2,0,1,0,0,...,0.000000,14.285714,42.857143,0.000000,28.571429,0.000000,14.285714,0.000000,0.000000,100.0
2009.0,2,4,1,1,1,2,0,2,2,2,...,23.529412,5.882353,5.882353,5.882353,11.764706,0.000000,11.764706,11.764706,11.764706,100.0
2010.0,0,7,1,1,3,4,2,0,2,5,...,28.000000,4.000000,4.000000,12.000000,16.000000,8.000000,0.000000,8.000000,20.000000,100.0


## no fund

In [23]:
data_no_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==0].set_index(['wos_ID','funding_yes'])
data_no_fund.head(2)

,,Publication Year,0,1,2,3,4,5,6,7,8,9
wos_ID,funding_yes,,,,,,,,,,,
wos:000904661800001,0,2023.0,0,0,0,1,0,0,0,0,0,0
wos:000849602200001,0,2022.0,0,0,0,0,0,0,0,0,1,0


In [24]:
data_topics_trend_no_fund = data_no_fund.groupby('Publication Year').sum()

data_topics_trend_no_fund['no_fund_total'] = data_topics_trend_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_no_fund.columns = data_topics_trend_no_fund.columns.astype(str)

for col in data_topics_trend_no_fund.columns:
    data_topics_trend_no_fund[col + '_%no_fund'] = data_topics_trend_no_fund[col]/data_topics_trend_no_fund['no_fund_total']*100
    

data_topics_trend_no_fund

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0


## concat and save

In [25]:
# concatenate dataframes on index
concat_data_topics_trend = pd.concat([data_topics_trend_all, 
                                    data_topics_trend_fund, 
                                    data_topics_trend_no_fund], 
                                    axis=1)
# save data
concat_data_topics_trend.to_csv(f'../TABLES/concat_data_topics_trend.csv')
concat_data_topics_trend

,0,1,2,3,4,5,6,7,8,9,...,1_%no_fund,2_%no_fund,3_%no_fund,4_%no_fund,5_%no_fund,6_%no_fund,7_%no_fund,8_%no_fund,9_%no_fund,no_fund_total_%no_fund
Publication Year,,,,,,,,,,,,,,,,,,,,,
1968.0,1,0,0,0,0,5,0,0,4,0,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,40.000000,0.000000,100.0
1969.0,0,0,0,1,0,2,0,0,1,0,...,0.000000,0.000000,25.000000,0.000000,50.000000,0.000000,0.000000,25.000000,0.000000,100.0
1970.0,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1971.0,0,0,0,0,0,1,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,50.000000,100.0
1972.0,0,0,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,100.0
1973.0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
1974.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1975.0,0,0,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,100.0
1977.0,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
